# Differential Privacy: Insurance Dataset

## 1) Introduction: Privacy–Utility Trade-off
This notebook studies the privacy–utility trade-off for a binary classifier predicting **smoker** status from the insurance dataset. We compare baseline models (SVM and decision tree) with feature-level differential privacy (DP) noise mechanisms and a DP-SGD neural network. **ROC-AUC** is the primary metric used throughout.


In [ ]:
from __future__ import annotations

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve

from dp.constants import RANDOM_STATE
from dp.dpsgd import opacus_available
from dp.mechanisms import add_gaussian_noise, add_laplace_noise
from dp.models import build_decision_tree_model, build_svm_model
from dp.pipeline import build_preprocessor, split_dataset

np.random.seed(RANDOM_STATE)
plt.style.use("seaborn-v0_8")


## 2) Dataset overview + class imbalance

In [ ]:
repo_root = Path("..").resolve()
data_path = repo_root / "data" / "insurance.csv"

df = pd.read_csv(data_path)
df.head()


In [ ]:
df.info()


In [ ]:
class_counts = df["smoker"].value_counts()
class_props = class_counts / class_counts.sum()

fig, ax = plt.subplots(figsize=(5, 4))
class_counts.plot(kind="bar", ax=ax)
ax.set_title("Class Imbalance (smoker)")
ax.set_ylabel("Count")
ax.set_xlabel("Class")
for idx, value in enumerate(class_counts.values):
    ax.text(idx, value, f"{class_props.iloc[idx]:.1%}", ha="center", va="bottom")
ax.grid(axis="y")
fig


## 3) Baseline models (SVM, Decision Tree)
We train baseline models with standard preprocessing (scaling numeric features and one-hot encoding categorical features). ROC-AUC is computed on a held-out test set.


In [ ]:
split = split_dataset(df, target="smoker")
preprocessor = build_preprocessor(split.X_train)

X_train = preprocessor.fit_transform(split.X_train)
X_test = preprocessor.transform(split.X_test)

y_train = (split.y_train == "yes").astype(int).to_numpy()
y_test = (split.y_test == "yes").astype(int).to_numpy()

models = {
    "svm": build_svm_model(),
    "decision_tree": build_decision_tree_model(),
}

baseline_rows = []
baseline_rocs = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    scores = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, scores)
    baseline_rows.append({"model": name, "roc_auc": roc_auc})
    fpr, tpr, _ = roc_curve(y_test, scores)
    baseline_rocs[name] = (fpr, tpr)

baseline_results = pd.DataFrame(baseline_rows)
baseline_results


In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
for name, (fpr, tpr) in baseline_rocs.items():
    auc_val = baseline_results.loc[baseline_results["model"] == name, "roc_auc"].iloc[0]
    ax.plot(fpr, tpr, label=f"{name} (AUC={auc_val:.3f})")
ax.plot([0, 1], [0, 1], linestyle="--", color="gray")
ax.set_title("Baseline ROC Curves")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.legend()
ax.grid(True)
fig


## 4) Feature-level DP
We apply DP noise to numeric features **after clipping** them to robust bounds. Clipping limits sensitivity, which controls the noise scale. We compare Laplace (ε-DP) and Gaussian ((ε, δ)-DP) mechanisms, then plot ROC-AUC vs ε on a log scale.


In [ ]:
def compute_clip_bounds(df: pd.DataFrame, lower_q: float = 0.01, upper_q: float = 0.99) -> pd.DataFrame:
    numeric = df.select_dtypes(include="number")
    return pd.DataFrame({
        "lower": numeric.quantile(lower_q),
        "upper": numeric.quantile(upper_q),
    })


def clip_numeric(df: pd.DataFrame, bounds: pd.DataFrame) -> pd.DataFrame:
    clipped = df.copy()
    for column in bounds.index:
        clipped[column] = clipped[column].clip(bounds.loc[column, "lower"], bounds.loc[column, "upper"])
    return clipped


def add_dp_noise(
    df: pd.DataFrame,
    mechanism: str,
    epsilon: float,
    delta: float,
    sensitivity: float,
    random_state: int,
) -> pd.DataFrame:
    numeric = df.select_dtypes(include="number")
    categorical = df.select_dtypes(exclude="number")
    if mechanism == "laplace":
        noisy_numeric = add_laplace_noise(numeric, epsilon=epsilon, sensitivity=sensitivity, random_state=random_state)
    elif mechanism == "gaussian":
        noisy_numeric = add_gaussian_noise(
            numeric,
            epsilon=epsilon,
            delta=delta,
            sensitivity=sensitivity,
            random_state=random_state,
        )
    else:
        raise ValueError("mechanism must be 'laplace' or 'gaussian'")
    return pd.concat([noisy_numeric, categorical], axis=1)[df.columns]


In [ ]:
bounds = compute_clip_bounds(split.X_train)

clipped_train = clip_numeric(split.X_train, bounds)
clipped_test = clip_numeric(split.X_test, bounds)

sensitivity = (bounds["upper"] - bounds["lower"]).max()

laplace_epsilons = np.logspace(-2, 1, 6)
gaussian_epsilons = np.logspace(-2, 1, 6)

def evaluate_dp_features(mechanism: str, epsilons: np.ndarray, delta: float = 1e-5) -> pd.DataFrame:
    rows = []
    for epsilon in epsilons:
        noisy_train = add_dp_noise(
            clipped_train,
            mechanism=mechanism,
            epsilon=epsilon,
            delta=delta,
            sensitivity=sensitivity,
            random_state=RANDOM_STATE,
        )
        preprocessor = build_preprocessor(noisy_train)
        X_train_noisy = preprocessor.fit_transform(noisy_train)
        X_test_noisy = preprocessor.transform(clipped_test)
        for name, model in models.items():
            model.fit(X_train_noisy, y_train)
            scores = model.predict_proba(X_test_noisy)[:, 1]
            roc_auc = roc_auc_score(y_test, scores)
            rows.append({"mechanism": mechanism, "epsilon": epsilon, "model": name, "roc_auc": roc_auc})
    return pd.DataFrame(rows)

laplace_results = evaluate_dp_features("laplace", laplace_epsilons)
gaussian_results = evaluate_dp_features("gaussian", gaussian_epsilons)

feature_dp_results = pd.concat([laplace_results, gaussian_results], ignore_index=True)
feature_dp_results.head()


In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
for (mechanism, model_name), subset in feature_dp_results.groupby(["mechanism", "model"]):
    ax.plot(subset["epsilon"], subset["roc_auc"], marker="o", label=f"{mechanism} - {model_name}")
ax.set_xscale("log")
ax.set_xlabel("Epsilon (log scale)")
ax.set_ylabel("ROC-AUC")
ax.set_title("Feature-level DP: ROC-AUC vs Epsilon")
ax.grid(True, which="both")
ax.legend()
fig


**Why performance collapses at very low ε:**
Low ε implies heavier noise, which overwhelms the signal in clipped numeric features. As the noise scale grows, discriminative patterns blur and classifiers approach random guessing (ROC-AUC ≈ 0.5). This is most pronounced for high-dimensional feature spaces where noise is injected per feature.


## 5) Excluded mechanisms
We exclude mechanisms such as **randomized response** (suited for binary categorical data) and **output perturbation** of model parameters. This notebook focuses on feature-level perturbation and training-time DP (DP-SGD) to keep the experimental scope clear and directly comparable via ROC-AUC.


## 6) DP-SGD (Neural Network)
We train a small neural network with **Opacus** to provide training-time privacy. We report ROC-AUC and the **computed ε** from the privacy accountant. We also note that **threshold tuning is post-processing**, which does not consume additional privacy budget.


In [ ]:
if opacus_available():
    import torch
    from torch import nn
    from torch.utils.data import DataLoader, TensorDataset
    from opacus import PrivacyEngine

    torch.manual_seed(RANDOM_STATE)

    X_full = preprocessor.fit_transform(split.X_train)
    X_eval = preprocessor.transform(split.X_test)
    y_full = torch.tensor(y_train, dtype=torch.float32)
    y_eval = torch.tensor(y_test, dtype=torch.float32)

    X_full_tensor = torch.tensor(X_full, dtype=torch.float32)
    X_eval_tensor = torch.tensor(X_eval, dtype=torch.float32)

    dataset = TensorDataset(X_full_tensor, y_full)

    def train_with_noise_multiplier(noise_multiplier: float, epochs: int = 15):
        model = nn.Sequential(
            nn.Linear(X_full_tensor.shape[1], 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
        loss_fn = nn.BCEWithLogitsLoss()
        data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

        privacy_engine = PrivacyEngine()
        model, optimizer, data_loader = privacy_engine.make_private(
            module=model,
            optimizer=optimizer,
            data_loader=data_loader,
            noise_multiplier=noise_multiplier,
            max_grad_norm=1.0,
        )

        model.train()
        for _ in range(epochs):
            for features, labels in data_loader:
                optimizer.zero_grad()
                logits = model(features).squeeze(1)
                loss = loss_fn(logits, labels)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            logits = model(X_eval_tensor).squeeze(1)
            probs = torch.sigmoid(logits).numpy()
        roc_auc = roc_auc_score(y_test, probs)
        epsilon = privacy_engine.get_epsilon(delta=1e-5)
        return epsilon, roc_auc

    noise_multipliers = [0.5, 1.0, 1.5, 2.0]
    dp_sgd_rows = []
    for nm in noise_multipliers:
        epsilon, roc_auc = train_with_noise_multiplier(nm)
        dp_sgd_rows.append({"noise_multiplier": nm, "epsilon": epsilon, "roc_auc": roc_auc})

    dp_sgd_results = pd.DataFrame(dp_sgd_rows)
    dp_sgd_results
else:
    dp_sgd_results = pd.DataFrame(columns=["noise_multiplier", "epsilon", "roc_auc"])
    print("Opacus/torch not available; skipping DP-SGD training.")


In [ ]:
if not dp_sgd_results.empty:
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(dp_sgd_results["epsilon"], dp_sgd_results["roc_auc"], marker="o")
    ax.set_xlabel("Computed epsilon")
    ax.set_ylabel("ROC-AUC")
    ax.set_title("DP-SGD: ROC-AUC vs Computed Epsilon")
    ax.grid(True)
    fig


**Threshold tuning (post-processing invariance):**
Once we have model scores, choosing a decision threshold is a post-processing step. Differential privacy is closed under post-processing, so tuning thresholds (e.g., for fairness or operational constraints) does **not** consume additional privacy budget.


## 7) Fairness baseline (DP diff, EO diff)
We report demographic parity (DP) difference and equalized odds (EO) difference using **sex** as the protected attribute. Metrics are computed on baseline model scores after choosing a threshold that maximizes Youden's J statistic on the test set.


In [ ]:
protected = split.X_test["sex"].to_numpy()

baseline_model = build_svm_model()
baseline_model.fit(X_train, y_train)
baseline_scores = baseline_model.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, baseline_scores)
J = tpr - fpr
best_threshold = thresholds[np.argmax(J)]
baseline_preds = (baseline_scores >= best_threshold).astype(int)


def demographic_parity_diff(y_pred: np.ndarray, group: np.ndarray) -> float:
    groups = np.unique(group)
    rates = []
    for g in groups:
        rates.append(y_pred[group == g].mean())
    return float(np.abs(rates[0] - rates[1]))


def equalized_odds_diff(y_true: np.ndarray, y_pred: np.ndarray, group: np.ndarray) -> float:
    groups = np.unique(group)
    tpr_diff = []
    fpr_diff = []
    for g in groups:
        mask = group == g
        positives = y_true[mask] == 1
        negatives = y_true[mask] == 0
        tpr_g = y_pred[mask][positives].mean() if positives.any() else 0.0
        fpr_g = y_pred[mask][negatives].mean() if negatives.any() else 0.0
        tpr_diff.append(tpr_g)
        fpr_diff.append(fpr_g)
    return float(max(np.abs(tpr_diff[0] - tpr_diff[1]), np.abs(fpr_diff[0] - fpr_diff[1])))

fairness_results = pd.DataFrame(
    {
        "metric": ["dp_diff", "eo_diff"],
        "value": [
            demographic_parity_diff(baseline_preds, protected),
            equalized_odds_diff(y_test, baseline_preds, protected),
        ],
    }
)

fairness_results


## 8) Discussion summary
- **Utility drops with stronger privacy**: As ε decreases, noise dominates feature signal and ROC-AUC trends toward chance-level.
- **Gaussian vs Laplace**: Both mechanisms follow the same trade-off pattern; Gaussian requires a δ parameter and can be tuned for similar utility.
- **DP-SGD provides training-time privacy**: When Opacus is available, the accountant yields an ε that can be compared directly to feature-level DP.
- **Fairness metrics**: DP/EO differences quantify group disparities and can be monitored alongside ROC-AUC without affecting privacy through post-processing.
